In [1]:
!pip install dagshub mlflow --quiet

import warnings
from statsmodels.tools.sm_exceptions import ValueWarning

warnings.filterwarnings("ignore", category=ValueWarning)
warnings.filterwarnings("ignore")

print("Done!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 64.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.5/95.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This be

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv
/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip


In [ ]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import mlflow
import mlflow.sklearn
import dagshub
import joblib
import os
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
from statsmodels.tsa.statespace.sarimax import SARIMAX

dagshub.init(repo_owner='gnada22', repo_name='ml_final_project', mlflow=True)

mlflow.set_experiment("SARIMA_Training")

In [4]:
class DateFeatureCreator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["week"] = (
            X["Date"].dt.to_period("W")
            .rank(method="dense")
            .astype(int) - 1
        )
        X["sin_13"] = np.sin(2 * np.pi * X["week"] / 13)
        X["cos_13"] = np.cos(2 * np.pi * X["week"] / 13)
        X["sin_23"] = np.sin(2 * np.pi * X["week"] / 23)
        X["cos_23"] = np.cos(2 * np.pi * X["week"] / 23)
        X = X.drop(columns=["Date"])
        return X

date_features = ["week", "sin_13", "cos_13", "sin_23", "cos_23"]

lag_features = []

class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        return X.drop(columns=self.columns, errors="ignore")

added_features = date_features + lag_features

class ColumnTransformerWithNames(ColumnTransformer):
    def get_feature_names_out(self, input_features=None):
        return super().get_feature_names_out(input_features)

    def transform(self, X):
        X_transformed = super().transform(X)
        cols = self.get_feature_names_out()
        cols = [c.split("__", 1)[-1] for c in self.get_feature_names_out()]
        res = pd.DataFrame(X_transformed, columns=cols, index=X.index)
        return res

    def fit_transform(self, X, y=None):
        X_transformed = super().fit_transform(X, y)
        cols = self.get_feature_names_out()
        cols = [c.split("__", 1)[-1] for c in self.get_feature_names_out()]
        res = pd.DataFrame(X_transformed, columns=cols, index=X.index)
        return res

class MultiIndexKeeper(BaseEstimator, TransformerMixin):
    def __init__(self, index_cols=["Date", "Store", "Dept"]):
        self.index_cols = index_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X.set_index(self.index_cols, drop=False, inplace=True)
        return X

In [6]:
class SARIMARegressor(BaseEstimator, RegressorMixin):
    def __init__(self, order=(1, 0, 0), seasonal_order=(1, 0, 0, 52), store_level='Store', dept_level='Dept'):
        self.order = order
        self.seasonal_order = seasonal_order
        self.store_level = store_level
        self.dept_level = dept_level

    def fit(self, X, y):
        if not isinstance(X.index, pd.MultiIndex):
            raise ValueError("X must have a MultiIndex")

        self.models_ = {}
        self.avgs_ = {}

        df = X.copy()
        df["target"] = y.values

        grouped = df.groupby(level=[self.store_level, self.dept_level])

        for (store, dept), group_df in grouped:
            # print("store, dept: ", store, dept)
            if dept == 1:
                print("store: ", store)
            group_df = group_df.sort_index(level="Date")

            dates = pd.to_datetime(group_df.index.get_level_values("Date"))
            dates = dates.sort_values()
            dates = pd.DatetimeIndex(dates)
            
            ts = group_df["target"].copy()
            exog = group_df.drop(columns=["target"])

            ts.index = dates
            exog.index = dates

            if ts is None or len(ts) < (self.order[0] + self.order[2] + 1) or ts.isna().all():
                print(f"Skipping Store {store}, Dept {dept} due to insufficient data")
                self.skip_(ts, store, dept)
                continue

            if ts is None or len(ts) < 3:
                self.skip_(ts, store, dept)
                continue
            
            ts = ts.dropna()
            
            # Skip if constant values (no variance)
            if ts.nunique() == 1:
                print(f"Skipping ({store}, {dept}) - constant series")
                self.skip_(ts, store, dept)
                continue
            
            # check standard deviation
            if ts.std() < 1e-6:
                print(f"Skipping ({store}, {dept}) - nearly constant series")
                self.skip_(ts, store, dept)
                continue
            try:
                model = SARIMAX(
                    endog=ts,
                    order=self.order,
                    seasonal_order=self.seasonal_order,
                    enforce_stationarity=False,
                    enforce_invertibility=False
                ).fit(disp=False)
                self.models_[(store, dept)] = model
            except (ValueError, np.linalg.LinAlgError) as e:
                # Skip problematic groups
                print(f"Skipping ({store}, {dept}) due to error: {e}")
                self.skip_(ts, store, dept)
                continue

        return self

    def skip_(self, ts, store, dept):
        if ts is None or len(ts) == 0:
            self.avgs_[(store, dept)] = 0.0
        else:
            self.avgs_[(store, dept)] = ts.mean()

    def predict(self, X):
        if not isinstance(X.index, pd.MultiIndex):
            raise ValueError("X must have a MultiIndex")
    
        preds = pd.Series(index=X.index, dtype=float)
    
        # Group X by store-dept pair (based on index levels)
        grouped = X.groupby(level=[self.store_level, self.dept_level])
    
        for (store, dept), group_df in grouped:
            if dept == 1:
                print("store: ", store)
            model = self.models_.get((store, dept))
            if model is None:
                preds.loc[group_df.index] = self.avgs_.get((store, dept), 0)
                continue

            group_df = group_df.sort_index(level="Date")

            dates = pd.to_datetime(group_df.index.get_level_values("Date"))
            dates = dates.sort_values()
            dates = pd.DatetimeIndex(dates)

            # exog = group_df.copy()
            # exog.index = dates
    
            # Forecast N steps = number of rows in this group
            forecast = model.forecast(steps=len(group_df))
            preds.loc[group_df.index] = forecast.to_numpy()
    
        return preds.to_numpy()

In [8]:
# Load and merge data
df_train = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip", parse_dates=["Date"])
df_features = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip", parse_dates=["Date"])
df_stores = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv")

df = df_train.merge(df_features, on=["Store", "Date", "IsHoliday"], how="left")
df = df.merge(df_stores, on="Store", how="left")

In [10]:
categorical = ["Store", "Dept", "Type", "IsHoliday"]
numerical = ["Temperature", "Fuel_Price", "CPI", "Unemployment",
             "MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5"]
engineered = added_features

categorical_transformer = OneHotEncoder(handle_unknown="ignore")
numerical_transformer = SimpleImputer(strategy="mean")

preprocessor = ColumnTransformerWithNames(
    transformers=[
        ("num", numerical_transformer, numerical + engineered),
        ("cat", categorical_transformer, categorical),
    ],
    sparse_threshold=0.0
)

pipeline = Pipeline([
    ("index_keeper", MultiIndexKeeper()),
    ("date_features", DateFeatureCreator()),
    ("preprocessor", preprocessor),
    ("column_dropper", ColumnDropper(columns=lag_features)),
    ("sarima", SARIMARegressor(order=(5,1,0)))
])

In [11]:
with mlflow.start_run(run_name="Feature_Engineering"):
    mlflow.log_param("features_added", added_features)

y = df["Weekly_Sales"]
X = df.drop(columns=["Weekly_Sales"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

🏃 View run Feature_Engineering at: https://dagshub.com/gnada22/ml_final_project.mlflow/#/experiments/4/runs/b759fd74d5b7438d99138c20efcc469f
🧪 View experiment at: https://dagshub.com/gnada22/ml_final_project.mlflow/#/experiments/4


In [12]:
# print(X_train.columns)

with mlflow.start_run(run_name="Training"):
    # grid_search.fit(X_train, y_train)
    # best_model = grid_search.best_estimator_
    # preds = best_model.predict(X_val)

    # mlflow.log_params(grid_search.best_params_)

    pipeline.fit(X_train, y_train)
    preds = pipeline.predict(X_val)
    best_model = pipeline

    mae = mean_absolute_error(y_val, preds)
    weights = X_val["IsHoliday"].apply(lambda x: 5 if x else 1)
    wmae = (weights * np.abs(y_val - preds)).sum() / weights.sum()

    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("WMAE", wmae)

    model_path = "model.pkl"
    joblib.dump(best_model, model_path)
    mlflow.log_artifact(model_path)
    
    # mlflow.sklearn.log_model(best_model, artifact_path="model")

store:  1
Skipping Store 1, Dept 77 due to insufficient data
Skipping Store 1, Dept 78 due to insufficient data
store:  2
Skipping Store 2, Dept 39 due to insufficient data
Skipping Store 2, Dept 77 due to insufficient data
store:  3
Skipping Store 3, Dept 77 due to insufficient data
Skipping Store 3, Dept 78 due to insufficient data
store:  4
store:  5
Skipping Store 5, Dept 51 due to insufficient data
Skipping Store 5, Dept 77 due to insufficient data
Skipping Store 5, Dept 78 due to insufficient data
Skipping Store 5, Dept 80 due to insufficient data
store:  6
🏃 View run Training at: https://dagshub.com/gnada22/ml_final_project.mlflow/#/experiments/4/runs/5a931476e49347919922fdbb9726ffc9
🧪 View experiment at: https://dagshub.com/gnada22/ml_final_project.mlflow/#/experiments/4


KeyboardInterrupt: 